In [ ]:
import pandas as pd
#file_path = './dataset/2023년_스포츠경기장주변_교통사고'
file_path = './dataset/2024년_스포츠경기장주변_교통사고'

file_path_csv = file_path + '.csv'
try:
    # 1. CSV 파일의 두 번째 줄을 읽어 컬럼명으로 사용
    with open(file_path_csv, 'r', encoding='utf-8-sig') as f: # utf-8-sig는 BOM(Byte Order Mark) 처리
        f.readline() # 첫 번째 줄 건너뛰기
        header_line = f.readline().strip() # 두 번째 줄 읽기
    correct_column_names = [col.strip() for col in header_line.split(',')]
    # BOM 문자가 첫 컬럼명에 붙어있을 수 있으므로 제거 (파일에서 직접 읽을 때 필요)
    if correct_column_names[0].startswith('\ufeff'):
        correct_column_names[0] = correct_column_names[0][1:]

    # 2. Pandas로 CSV 데이터 읽기
    #    - skiprows=2: 파일의 처음 두 줄(헤더들)을 건너뛰고 데이터부터 읽음
    #    - header=None: 파일 자체에 헤더가 없다고 Pandas에 알림 (우리가 직접 지정할 것이므로)
    #    - names=correct_column_names: 위에서 추출한 컬럼명 사용
    #    - encoding='utf-8-sig': 한글 깨짐 및 BOM(Byte Order Mark) 처리
    #    - na_values='-': '-' 값을 NaN으로 처리
    df = pd.read_csv(file_path_csv, skiprows=2, header=None, names=correct_column_names, encoding='utf-8-sig', na_values='-')

    # 3. 마지막 행이 전부 NaN 값인 경우 제거 (주로 CSV 마지막에 빈 줄이 있을 때 발생)
    if not df.empty and df.iloc[-1].isnull().all():
        df = df.iloc[:-1].copy() # .copy()를 사용하여 SettingWithCopyWarning 방지

    if df.empty:
        print(f"'{file_path_csv}' 파일에서 데이터를 읽어오지 못했거나, 데이터가 비어있습니다.")
    else:
        # 4. '구분' 컬럼 생성 (사고, 사망, 부상)
        # '사고년도' 컬럼에 실제 구분값이 들어있음
        df['구분'] = df['사고년도'].str.replace(r'\[(건|명)\]', '', regex=True)

        # 5. 데이터 Melt (Wide to Long)
        id_cols = ['시도', '시군구', '사고월', '구분']
        day_cols = [col for col in df.columns if col.endswith('일') and col[:-1].isdigit()]
        df_melted = pd.melt(df, id_vars=id_cols, value_vars=day_cols, var_name='일자_컬럼명', value_name='값')

        # 6. 값이 없는 행(원본에서 '-'였던 부분) 제거
        df_melted.dropna(subset=['값'], inplace=True)
        df_melted['값'] = pd.to_numeric(df_melted['값']) # 값 컬럼을 숫자형으로 변환

        # 7. '날짜' 컬럼 생성
        df_melted['월_숫자'] = df_melted['사고월'].str.replace('월', '').astype(int)
        df_melted['일_숫자'] = df_melted['일자_컬럼명'].str.replace('일', '').astype(int)

        # 날짜 변경 
        #df_melted['날짜'] = pd.to_datetime(
        #    '2023-' + df_melted['월_숫자'].astype(str).str.zfill(2) + '-' + df_melted['일_숫자'].astype(str).str.zfill(2),
        #    errors='coerce' # 유효하지 않은 날짜는 NaT로 변환
        #)
        df_melted['날짜'] = pd.to_datetime(
            '2024-' + df_melted['월_숫자'].astype(str).str.zfill(2) + '-' + df_melted['일_숫자'].astype(str).str.zfill(2),
            errors='coerce' # 유효하지 않은 날짜는 NaT로 변환
        )
        df_melted.dropna(subset=['날짜'], inplace=True) # NaT가 된 행(예: 2월 30일) 제거
        df_melted['날짜'] = df_melted['날짜'].dt.strftime('%Y.%m.%d')


        # 8. Pivot_table을 사용하여 '구분'을 컬럼으로 변환
        df_pivoted = df_melted.pivot_table(
            index=['시도', '시군구', '날짜'],
            columns='구분',
            values='값',
            aggfunc='first' # 일반적으로 중복이 없으므로 first 사용
        ).reset_index()

        # pivot_table로 생성된 컬럼 이름의 name 속성 제거
        df_pivoted.columns.name = None

        # 9. '지역' 컬럼 생성 및 최종 컬럼 선택/정리
        df_pivoted['지역'] = df_pivoted['시도'] + ' ' + df_pivoted['시군구']

        # 필요한 컬럼만 선택하고 순서 지정
        final_df = df_pivoted[['지역', '날짜', '사고', '사망', '부상']].copy() # .copy() 추가

        # 데이터 타입 변환 (정수형, 결측치 허용)
        for col in ['사고', '사망', '부상']:
            final_df[col] = final_df[col].astype('Int64')

        # 10. 결과 정렬
        final_df = final_df.sort_values(by=['지역', '날짜']).reset_index(drop=True)

        file_path_transformed = file_path + '_transformed.csv'
        final_df.to_csv(file_path_transformed, index=False, encoding='utf-8-sig')
        
        print("최종 변환된 DataFrame:")
        print(final_df.head(10))
        print("\nDataFrame 정보:")
        final_df.info()

except FileNotFoundError:
    print(f"오류: 파일 '{file_path}'을(를) 찾을 수 없습니다. 파일 경로를 확인해주세요.")
except Exception as e:
    print(f"데이터 처리 중 오류가 발생했습니다: {e}")

최종 변환된 DataFrame:
       지역          날짜  사고  사망  부상
0  강원 강릉시  2024.01.01   3   0   3
1  강원 강릉시  2024.01.02   2   0   2
2  강원 강릉시  2024.01.04   2   0   2
3  강원 강릉시  2024.01.05   2   0   2
4  강원 강릉시  2024.01.06   4   0   4
5  강원 강릉시  2024.01.07   2   0   3
6  강원 강릉시  2024.01.08   3   0   4
7  강원 강릉시  2024.01.09   2   1   1
8  강원 강릉시  2024.01.10   1   0   1
9  강원 강릉시  2024.01.11   2   0   2

DataFrame 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17040 entries, 0 to 17039
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   지역      17040 non-null  object
 1   날짜      17040 non-null  object
 2   사고      17040 non-null  Int64 
 3   사망      17040 non-null  Int64 
 4   부상      17040 non-null  Int64 
dtypes: Int64(3), object(2)
memory usage: 715.7+ KB
